In [9]:
import dlib 
import cv2 as cv

In [10]:
detector = dlib.get_frontal_face_detector()
eye_cascade = cv.CascadeClassifier(cv.data.haarcascades + 'haarcascade_eye.xml')

In [12]:
import dlib

# Load the Dlib face detector
detector = dlib.get_frontal_face_detector()

def getextractedeyes_dlib(Path, target_resolution=(145, 145)):
    img = cv.imread(Path)
    name = Path[-13:-8]
    print(name)
    gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    ans = []

    # Use Dlib face detector
    faces = detector(gray_img)

    if len(faces) > 0:
        for i, face in enumerate(faces):
            x, y, w, h = face.left(), face.top(), face.width(), face.height()
            roi_gray = gray_img[y:y+h, x:x+w]
            roi_color = img[y:y+h, x:x+w]

            eyes = eye_cascade.detectMultiScale(roi_gray)

            for j, (ex, ey, ew, eh) in enumerate(eyes):
                eye = roi_color[ey:ey+eh, ex:ex+ew]
                eye = cv.resize(eye, target_resolution)
                ans.append(eye)

                # Determine the eye side (left or right)
                eye_side = "left" if j % 2 == 0 else "right"

                # Save the resized eye image
                respath = f"D:/projects/Driver_Drowziness/results/{name}_{eye_side}_eye_{i + 1}_{j + 1}.jpg"
                cv.imwrite(respath, eye)
    else:
        print(f"No faces detected in {Path}")

    return ans


In [13]:
Path = r"C:\Users\vkris\Pictures\Camera Roll\WIN_20231121_23_52_35_Pro.jpg"
eyes = getextractedeyes_dlib(Path)

52_35
No faces detected in C:\Users\vkris\Pictures\Camera Roll\WIN_20231121_23_52_35_Pro.jpg


In [14]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np

In [15]:
model = load_model(r"D:\projects\Driver_Drowziness\drowsiness_detection_model_final.h5")

In [16]:
def preprocess_eye_image(img):
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    return img_array

In [8]:
lefteye = False
righteye = False
a = 0
for i in eyes:
    eye = preprocess_eye_image(i)
    pred = model.predict(eye)
    print(pred)
    if pred[0, 0] > 0.5:
        if (a==0): lefteye = False
        else: righteye = False
    else:
        if (a==0): lefteye = True
        else: righteye = True
    a+=1
if (lefteye==True and righteye==True): print("True, drowsiness detected!")
else: print("No Drowsiness Detected")

TypeError: 'NoneType' object is not iterable